Improve the data and exclude unnecessary information

In [ ]:
import pandas as pd


In [ ]:
#orders.csv
url = "https://drive.google.com/file/d/1VpCRIiyfM4uF6EN6KZw1E0_sAGeBhvMI/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1i_tpUVt3B7RLU5o-4rP_BKHsE_8rNVya/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)

# products.csv
url = "https://drive.google.com/file/d/1vuIA7mbbyurdpKXIIuLsxLttCweHIKJU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

# brands.csv
url = "https://drive.google.com/file/d/1badESik7s10CrZhD9_dp23g_F1SVazV4/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)

In [ ]:
new_products = products.copy()
new_orders = orders.copy()
new_orderlines = orderlines.copy()
new_brands = brands.copy()


Adding some useful columns for different tables


In [ ]:

#Holiday season table for new_orders
new_orders['created_date'] = pd.to_datetime(new_orders['created_date'])
new_orders['its_time'] = {}

def holiday (date):
    if date in [11,12,1]:
         return 'High Season'
    else:
      return 'Low Season'

new_orders['its_time'] = new_orders['created_date'].dt.month.apply(holiday)

# add percentage to products
new_products['price'] = new_products['price'].round(2)
new_products['promo_percentage'] = ((1 - (new_products['corrected_promo_price'] / new_products['price'])) * 100).round(2)

 #new column for total price
new_orderlines['total_price'] = new_orderlines.unit_price * new_orderlines.product_quantity

In [ ]:
new_products

,sku,name,desc,price,promo_price,in_stock,type,corrected_promo_price,promo_percentage
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,49.99,16.67
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,589.996,0,13855401,59.00,0.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,569.898,0,1387,56.99,3.41
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,229.997,0,1230,23.00,8.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.990,1,1364,31.99,8.57
...,...,...,...,...,...,...,...,...,...
10529,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282,26.99,10.00
10530,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,64.99,7.09
10531,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,64.99,7.09
10532,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392,64.99,7.09


Exclude outliers

In [ ]:
new_products = new_products.dropna()
new_orders = new_orders.loc[new_orders['state'] == 'Completed',:] #orders only for state "completed"
#match orders that are in the products table
new_orderlines = new_orderlines.loc[new_orderlines['sku'].isin(new_products['sku']),:]
new_orders = new_orders.loc[new_orders['order_id'].isin(new_orderlines['id_order']),:]
new_orderlines = new_orderlines.loc[new_orderlines['id_order'].isin(new_orders['order_id']),:]

The column total_price from orders doesn't match the price from the other tables for each order_id

It could be due to shipping cost.

To do:


*   Find the Q1, Q3 and IQR for the price diff
*   Exclude the outliers





In [ ]:
new_orderlines

,id,id_order,product_id,product_quantity,sku,unit_price,date,total_price
7,1119116,299545,0,1,OWC0100,47.49,2017-01-01 01:46:16,47.49
8,1119119,299546,0,1,IOT0014,18.99,2017-01-01 01:50:34,18.99
9,1119120,295347,0,1,APP0700,72.19,2017-01-01 01:54:11,72.19
11,1119126,299549,0,1,PAC0929,2565.99,2017-01-01 02:07:42,2565.99
18,1119134,299556,0,1,CRU0039-A,60.90,2017-01-01 02:20:14,60.90
...,...,...,...,...,...,...,...,...
293599,1649474,525664,0,1,TUC0207,16.52,2018-03-14 11:45:05,16.52
293616,1649512,527070,0,2,APP0698,9.99,2018-03-14 11:49:01,19.98
293621,1649522,527074,0,2,APP0698,9.99,2018-03-14 11:49:36,19.98
293644,1649565,527096,0,3,APP0698,9.99,2018-03-14 11:54:35,29.97


In [ ]:
new_orderlines_id = new_orderlines.groupby(new_orderlines.id_order,as_index=False)['total_price'].sum()
new_orders_lines = new_orderlines.merge(new_orders, left_on='id_order',right_on='order_id',how='inner')

order_diff = new_orders.merge(new_orderlines_id,left_on='order_id',right_on='id_order',how='inner')

order_diff['price_diff'] = order_diff.total_paid - order_diff.total_price
order_diff.describe()

q1 = order_diff['price_diff'].quantile(0.25)
q3 = order_diff['price_diff'].quantile(0.75)
iqr = q3 - q1

diff_no_outliers_df = order_diff.loc[
    (order_diff["price_diff"] >= (q1 - 1.5*iqr))
    &
    (order_diff["price_diff"] <= (q3 + 1.5*iqr))]
no_outliers_id = diff_no_outliers_df['order_id']

new_orders = new_orders.loc[new_orders['order_id'].isin(no_outliers_id),:]
new_orderlines = new_orderlines.loc[new_orderlines['id_order'].isin(no_outliers_id),:]




Improving product categorization

In [ ]:
new_products

,sku,name,desc,price,promo_price,in_stock,type,corrected_promo_price,promo_percentage
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696,49.99,16.67
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59.00,589.996,0,13855401,59.00,0.00
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59.00,569.898,0,1387,56.99,3.41
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25.00,229.997,0,1230,23.00,8.00
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.990,1,1364,31.99,8.57
...,...,...,...,...,...,...,...,...,...
10529,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.903,1,12282,26.99,10.00
10530,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,64.99,7.09
10531,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.903,1,1392,64.99,7.09
10532,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.903,0,1392,64.99,7.09


In [ ]:
#Remove unwanted columns

new_products =  new_products.drop('promo_price', axis=1)
new_products =  new_products.drop('corrected_promo_price', axis=1)
new_products =  new_products.drop('promo_percentage', axis=1)
#adding new categories
new_products['category'] = ""

new_products.loc[(new_products['desc'].str.contains('iphone|ipad|mac|apple|mac|mouse|keyboard',case=False)) & (new_products['price'] < 400) ,'category'] = 'Apple Accesories'
new_products.loc[(new_products['name'].str.contains('iphone|ipad|mac|apple|mac|mouse|keyboard',case=False)) & (new_products['price'] < 400) ,'category'] = 'Apple Accesories'
new_products.loc[(new_products['name'].str.contains('case',case=False)) & (new_products['price'] < 130) ,'category'] = 'Cases'
new_products.loc[(new_products['name'].str.contains('drive|ssd|raid|memory card',case=False)) | (new_products['desc'].str.contains('drive|ssd|raid|nas|memory card',case=False)),'category'] = 'Storage'
new_products.loc[(new_products['price'] > 1000) & (new_products['name'].str.contains('imac|mac pro|mac mini',case=False)),'category'] = 'Desktop'
new_products.loc[(new_products['price'] > 1000) & (new_products['name'].str.contains('macbook',case=False)),'category'] = 'Laptop'
new_products.loc[new_products['type'] == '11821715','category'] = 'Ipod'
new_products.loc[(new_products['price'] > 300) & (new_products['name'].str.contains('ipad',case=False)),'category'] = 'Tablet'
new_products.loc[(new_products['type'] != '1,44+11') & (new_products['name'].str.contains('headset|headphones|in-ear',case=False)),'category'] = 'Headphones'
new_products.loc[new_products['name'].str.contains('router',case=False),'category'] = 'Routers'
new_products.loc[(new_products['price'] > 150) & new_products['name'].str.contains('watch|smartwatch',case=False),'category'] = 'Watches'
new_products.loc[(new_products['name'].str.contains ("apple iphone",case=False)) & (new_products['price'] > 400),'category'] = 'Smartphones'
new_products.loc[new_products['sku'].str.contains('ALL|KEN|LIN|SAN|WDT'),'category'] = 'IT Accesories'
new_products.loc[new_products.name.str.contains('monitor',case=False),'category'] = 'Monitors'
new_products.loc[new_products['type'].str.contains('1,44+'),'category'] = 'Repair'
new_products.loc[new_products['name'].str.contains ("speaker",case=False),'catnewegory'] = 'Speakers'
new_products.loc[new_products.category == "",'category'] = 'other'
new_products['short'] = new_products['sku'].str[:3]
new_products = new_products.merge(brands, on='short',how='inner')
new_products = new_products.drop('short',axis=1)
new_products = new_products.drop('catnewegory',axis=1)
new_products.loc[new_products["desc"].str.contains("refurbished|reconditioned|like new", case=False), "category"] += ", refurbished"
new_products.loc[new_products["name"].str.contains("^.{0,7}apple ipod", case=False), "category"] += ", ipod"

In [ ]:
new_products

NameError: name 'new_products' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
new_products.to_csv('/content/drive/My Drive/Datasets/products_cl.csv', index=False)
new_orders.to_csv('/content/drive/My Drive/Datasets/orders_cl.csv', index=False)
new_orderlines.to_csv('/content/drive/My Drive/Datasets/orderlines_cl.csv', index=False)
new_brands.to_csv('/content/drive/My Drive/Datasets/brands_cl.csv', index=False)